In [7]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tqdm
!pip install gdown
!pip install -U ckip-transformers

In [8]:
# Download data here: https://drive.google.com/file/d/1yyqTwl3SCMQ6P6ZWt88cPqvtzK4RdI2m/view?usp=sharing

root = '/content/drive/MyDrive/Colab Notebooks/QABot'
data_path = root + '/wiki_2021_08_05_1215639.json'


In [9]:
import json
with open(data_path, 'r', encoding='UTF-8') as f:
	data = json.load(f)

print(data[0])
print(len(data))

{'id': 1, 'title': '福斯家庭電影頻道', 'articles': '福斯家庭電影頻道（）是一個播放電影的亞洲電視頻道，由福斯國際頻道持有與出品，STAR衛視發佈。在香港、澳門、臺灣、菲律賓、新加坡、馬來西亞等地區均有業務。福斯家庭電影頻道已簽訂電影合約的單位包括20世紀福斯、迪斯尼、米高梅 - 梅耶，Studio Cana、獅門娛樂、頂峯娛樂、溫斯坦公司等。該電影頻道面向老人和兒童，主要提供家庭電影。'}
1215639


In [10]:
from ckip_transformers.nlp import CkipWordSegmenter
from tqdm import tqdm
import re

In [ ]:
ws_driver = CkipWordSegmenter(level=1, device=0)

In [ ]:
batch_size = 100
chunk_size = 48
reg = "[\s\-，.。\:：！!；;\?？（）\(\)\"\'《》〈〉．～—─\=「」『』、”“·／\#\[\]\$,－]"

# reference: https://github.com/ckiplab/ckip-transformers
# reference: https://colab.research.google.com/github/alvinntnu/NTNU_ENC2045_LECTURES/blob/main/nlp/chinese-word-seg.ipynb#scrollTo=RTgmrVLm9teO

skip_chunk = 0      # from the n-th chunk you want to generate.

for index in tqdm(range(len(data) // batch_size)):
    if index < skip_chunk*chunk_size:
        continue

    if index % chunk_size == 0:
        f = open('%s/output-%07d-%07d.txt' %
                 (root, index*batch_size, (index+chunk_size)*batch_size-1), 'w', encoding='UTF-8')

    input = []
    for i in range(batch_size):
        input.append(re.sub(reg, " ", data[index*batch_size+i]['articles'] + " " + data[index*batch_size+i]['title']))

    text = ws_driver(input, show_progress=False, batch_size = batch_size)

    for i in range(batch_size):
        write = str(data[index*batch_size+i]['id']) + ' '
        for key in text[i]:
            key = key.strip()
            if key != '':
                write += key + ' '
        write += '\n'
        f.write(write)

    if index % chunk_size == chunk_size-1:
        print("success! %d" % (index))
        f.close()